In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(20,5)
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pyarrow.parquet as pq

In [2]:
dframe=pd.read_parquet("d:/Data_Files/DLMS Load Profile/data.parquet")

In [3]:
dframe['Kwh']=dframe['Kwh']/1000
id=dframe['sensor'].unique()


In [4]:
dframe = dframe.groupby('sensor')

In [10]:
lst=[]
for i in dframe:
    df=i
    print(df)
    print(type(df))
    lst.append(df)

('5f718c439c7a78.65267835',                         sensor               Clock  R_Voltage  Y_Voltage  \
0      5f718c439c7a78.65267835 2022-11-18 00:30:00     6499.0     6514.0   
1      5f718c439c7a78.65267835 2022-11-18 01:00:00     6520.0     6535.0   
2      5f718c439c7a78.65267835 2022-11-18 01:30:00     6536.0     6551.0   
3      5f718c439c7a78.65267835 2022-11-18 02:00:00     6576.0     6591.0   
4      5f718c439c7a78.65267835 2022-11-18 02:30:00     6590.0     6605.0   
...                        ...                 ...        ...        ...   
16703  5f718c439c7a78.65267835 2023-11-01 00:30:00     6426.0     6451.0   
16704  5f718c439c7a78.65267835 2023-11-01 01:00:00     6447.0     6473.0   
16705  5f718c439c7a78.65267835 2023-11-01 01:30:00     6473.0     6497.0   
16706  5f718c439c7a78.65267835 2023-11-01 02:00:00     6491.0     6512.0   
16717  5f718c439c7a78.65267835 2023-10-31 23:30:00     6328.0     6357.0   

       B_Voltage  R_Current  Y_Current  B_Current    Kwh  


In [12]:
df=lst[0]

In [13]:
df.head()

AttributeError: 'tuple' object has no attribute 'head'

In [9]:
df

('62a9920f75c931.62399458',
                         sensor               Clock  R_Voltage  Y_Voltage  \
 16707  62a9920f75c931.62399458 2022-05-13 00:30:00     6094.0     6090.0   
 16708  62a9920f75c931.62399458 2022-05-13 01:00:00     6154.0     6144.0   
 16709  62a9920f75c931.62399458 2022-05-13 01:30:00     6189.0     6186.0   
 16710  62a9920f75c931.62399458 2022-05-13 02:00:00     6219.0     6211.0   
 16711  62a9920f75c931.62399458 2022-05-13 02:30:00     6249.0     6241.0   
 ...                        ...                 ...        ...        ...   
 38361  62a9920f75c931.62399458 2023-11-01 10:00:00     6021.0     6006.0   
 38362  62a9920f75c931.62399458 2023-11-01 10:30:00     6009.0     5995.0   
 38363  62a9920f75c931.62399458 2023-11-01 11:00:00     6024.0     6012.0   
 38364  62a9920f75c931.62399458 2023-11-01 11:30:00     6029.0     6016.0   
 38365  62a9920f75c931.62399458 2023-11-01 12:00:00     6019.0     6007.0   
 
        B_Voltage  R_Current  Y_Current  B_Cur

In [ ]:
lst=[]
dframe = dframe.groupby('sensor')
for each_id  in dframe:
    df=each_id


for location in id:
    df=dframe[dframe['sensor'] == location ]
    # df['Clock']=pd.to_datetime(df['Clock'])
    df.set_index(['Clock'],inplace=True,drop= True)
    df = df[df.index >= '2022-11-18 00:00:00']
    filtered_df = df[((df['R_Voltage'] == 0) | (df['Y_Voltage'] == 0) | (df['B_Voltage'] == 0)) & 
                     ((df['R_Current'] == 0) | (df['Y_Current'] == 0) | (df['B_Current'] == 0))]
    filtered_df['Kwh'] = 0
    df.loc[filtered_df.index, :] = filtered_df
    df=df[['Kwh']].resample(rule='1H').sum()
    # df['id']=location
    df['Kwh_r']=df['Kwh'].rolling(window=24).mean()
    df.dropna(inplace=True)

    lst.append(df)

In [19]:
lst=[]
sensor_groups = dframe.groupby('sensor')

# Iterate over groups
for location, group_df in sensor_groups:
    print(group_df)
    print(type(group_df))
    # 'location' will be the sensor value, and 'group_df' will be the corresponding DataFrame
    # You can perform operations on 'group_df' here
    # If you want to append the result to a list, you can do something like:
    lst.append(group_df)

                        sensor               Clock  R_Voltage  Y_Voltage  \
0      5f718c439c7a78.65267835 2022-11-18 00:30:00     6499.0     6514.0   
1      5f718c439c7a78.65267835 2022-11-18 01:00:00     6520.0     6535.0   
2      5f718c439c7a78.65267835 2022-11-18 01:30:00     6536.0     6551.0   
3      5f718c439c7a78.65267835 2022-11-18 02:00:00     6576.0     6591.0   
4      5f718c439c7a78.65267835 2022-11-18 02:30:00     6590.0     6605.0   
...                        ...                 ...        ...        ...   
16703  5f718c439c7a78.65267835 2023-11-01 00:30:00     6426.0     6451.0   
16704  5f718c439c7a78.65267835 2023-11-01 01:00:00     6447.0     6473.0   
16705  5f718c439c7a78.65267835 2023-11-01 01:30:00     6473.0     6497.0   
16706  5f718c439c7a78.65267835 2023-11-01 02:00:00     6491.0     6512.0   
16717  5f718c439c7a78.65267835 2023-10-31 23:30:00     6328.0     6357.0   

       B_Voltage  R_Current  Y_Current  B_Current     Kwh  
0         6473.0      27.74

In [16]:
sensor_groups

In [ ]:

df=lst[0]
df
df['Kwh'].describe()

result=adfuller(df['Kwh'],autolag='AIC')
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])



In [20]:
import pandas as pd

# Creating a sample DataFrame
data = {'sensor': ['A', 'B', 'A', 'B', 'A', 'B'],
        'value': [1, 2, 3, 4, 5, 6]}
df = pd.DataFrame(data)

# Grouping by the 'sensor' column
grouped = df.groupby('sensor')

# Iterating over groups
for name, group in grouped:
    print(f"Sensor: {name}")
    print(group)

Sensor: A
  sensor  value
0      A      1
2      A      3
4      A      5
Sensor: B
  sensor  value
1      B      2
3      B      4
5      B      6
